In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
from numpy import linalg as la
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# http://www.hpcc.unn.ru/?dir=1052

class Jacobi:
    """
    n - dimension of system
    """
    def __init__(self, n):
        self.n = n
        def pos_semidef_mtx():
            tmp_mtx = np.random.randint(4, size=(self.n, self.n))
            return np.dot(tmp_mtx, tmp_mtx.T)
        self.A = pos_semidef_mtx()
        for i in range(self.n):
            tmp = 0
            for j in range(self.n):
                tmp += self.A[i,j]
            self.A[i,i] += tmp
        self.b = np.random.rand(self.n)
        def alpha_mat():
            alpha_mtx = np.zeros((self.n, self.n))
            for i in range(0, self.n):
                for j in range(0, self.n):
                    if i != j:
                        alpha_mtx[i][j] = - self.A[j][i] / self.A[i][i]
            return alpha_mtx
        self.alpha = alpha_mat()
        def beta_vec():
            return self.b / np.diag(self.A)
        self.beta = beta_vec()
        
        def check_diag_nonzero(mat):
            return all(x != 0 for x in np.diag(mat))
        assert(check_diag_nonzero(self.A) == 1)
    
    def solve(self, tolerance=1e-10):
        assert(tolerance > 0)
        x_prev = self.beta
        converged = 0
        for i in range(1000):
            x_next = np.dot(self.alpha, x_prev.T) + self.beta
            if max(abs(x_prev - x_next)) < tolerance:
                converged = 1
                break
            x_prev = x_next
        return (x_next, i, converged)
    
    def linalg_solve(self):
        return np.linalg.solve(self.A, self.b)
    
    def calc_error(self, tolerance=1e-10):
        return spatial.distance.euclidean(self.solve(tolerance)[0], self.linalg_solve())
    
    def get_eigen_vals(self):
        return la.eigvals(self.A)
    
    def __repr__(self):
        return ("A: {}\n\n"
                "b: {}\n\n"
                "alpha:\n {}\n\n"
                "beta: {}".format(self.A, self.b, self.alpha, self.beta))
        
    def __str__(self):
        return ("A: {}\n\n"
                "b: {}\n\n"
                "alpha:\n {}\n\n"
                "beta: {}".format(self.A, self.b, self.alpha, self.beta))

In [5]:
j = Jacobi(5)
print(j)
sol = j.solve()
print('<------|------>\n')
print('Собственные числа:', j.get_eigen_vals())
print('Точное решение:              ', np.linalg.solve(j.A, j.b))
print('Решение итерационным методом:', sol[0])
print('Число итераций:', sol[1])
print('Точная ошибка:', j.calc_error())

A: [[156  24  15  24  21]
 [ 24 120  15  18  15]
 [ 15  15  95  20   9]
 [ 24  18  20 126  12]
 [ 21  15   9  12  93]]

b: [ 0.11483713  0.29593421  0.7857676   0.3876887   0.66832797]

alpha:
 [[ 0.         -0.15384615 -0.09615385 -0.15384615 -0.13461538]
 [-0.2         0.         -0.125      -0.15       -0.125     ]
 [-0.15789474 -0.15789474  0.         -0.21052632 -0.09473684]
 [-0.19047619 -0.14285714 -0.15873016  0.         -0.0952381 ]
 [-0.22580645 -0.16129032 -0.09677419 -0.12903226  0.        ]]

beta: [ 0.00073614  0.00246612  0.00827124  0.00307689  0.00718632]
<------|------>

Собственные числа: [ 199.43467053  117.45854494  104.82976204   84.98858532   83.28843716]
Точное решение:               [-0.00117625  0.00075846  0.00743327  0.00140037  0.00642955]
Решение итерационным методом: [-0.00117625  0.00075846  0.00743327  0.00140037  0.00642955]
Число итераций: 32
Точная ошибка: 7.867736228605049e-11
